In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import json
from datetime import datetime
import time
import re
from bs4 import BeautifulSoup

In [2]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36')
driver = webdriver.Chrome(options=options)
actions = ActionChains(driver)
driver.implicitly_wait(2)

In [ ]:
def parse_with_soup(html):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    except Exception as e:
        print(f"파싱 실패: {e}")
        return None

def load_test_page():
    driver.get("https://kmong.com/category/661")
    print("모달창 생략을 위한 테스트 페이지 로드 완료")
    time.sleep(1)

def parse_service_amount():
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'css-enj2mu')))
    service_amount = int(driver.find_element(By.CLASS_NAME, "css-enj2mu").text.replace("개의 서비스",""))
    return service_amount

def parse_service_infos(soup):
    service_list = soup.find_all("article", "edqw2x10")

    parse_list = []
    for service in service_list:
        title = service.find("span", "text-[14px] font-bold leading-[21px] text-gray-900 mb-1 line-clamp-2").text
        seller = service.find("span", "line-clamp-1 text-xs font-normal leading-[18px] text-gray-600").text
        link = service.find("a").attrs['href']
        # print(title, seller, link)
        parse_list.append([title, seller, link])

    return parse_list

def crawl_services(category_id):
    load_test_page()
    driver.get(f"https://kmong.com/category/{category_id}")

    service_list = []
    page_idx = 1
    service_amount = parse_service_amount()

    while True:
        print(f"Page {page_idx} : ", end="")

        # 서비스 목록 파싱
        WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'edqw2x10')))

        html = driver.page_source
        soup = parse_with_soup(html)

        parse_list = parse_service_infos(soup)
        service_list.extend(parse_list)
        print(f"[{len(service_list)}/{service_amount}]")

        try:
            box = driver.find_element(By.CLASS_NAME, "e1t3wbc50")
            next_page_btn = box.find_elements(By.TAG_NAME, "button")[-1]
            if next_page_btn.is_enabled():
                next_page_btn.click()
            else:
                return service_amount, service_list
        except Exception as e:
            print("단일 페이지입니다.")
            return service_amount, service_list

        time.sleep(2)
        page_idx += 1

모달창 생략을 위한 테스트 페이지 로드 완료
Page 1 : [0/373]
Page 2 : [36/373]
Page 3 : [72/373]
Page 4 : [108/373]
Page 5 : [144/373]
Page 6 : [180/373]
Page 7 : [216/373]
Page 8 : [252/373]
Page 9 : [288/373]
Page 10 : [324/373]
Page 11 : [360/373]
